In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.style
matplotlib.style.use("seaborn")
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sn
sn.color_palette("hls", 17)
import scipy.stats as st
import math

from pingouin import rcorr

In [2]:
import sys
import os

from sys import platform
if platform == "darwin":
    sys.path.append(os.path.dirname(os.path.realpath(__file__)) + "/..")
    smart_nlp_path = ''
elif platform == "win32":
    sys.path.append('../')
    smart_nlp_path = os.getcwd()
    smart_nlp_path = "\\".join([smart_nlp_path.split("\\")[i] for i in range(0,len(smart_nlp_path.split("\\"))-1)]+["/"])

from module.trend_analysis_functions import *
from module.topic_model_plus_class import Topic_Model_plus

In [3]:
csv_file_name = smart_nlp_path+r"/input data/safecom-2011-present-NASA.xlsx" 
df = pd.read_excel(csv_file_name,sheet_name='safecom-2011-present-NASA')
df['fiscal_year'].value_counts()

C:\Users\srandrad\Anaconda3\lib\site-packages\openpyxl\compat\numbers.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.float,


2021    2543
2020    1392
2016     998
2018     943
2017     894
2015     714
2019     611
2014       3
2009       1
Name: fiscal_year, dtype: int64

In [4]:
list_of_attributes = ['narr_public']#, 'corrective_public', 'notes']
extra_cols = ['region', 'agency', 'duplicate_yn', 'completed_yn', 'rep_by_org',
                   'air_number', 'air_type', 'air_model', 'air_manufacturer',
                   'air_owner', 'mission_destination', 'mission_depart', 'mission_hazmat',
                   'mission_special_use', 'mission_pax', 'mission_procurement_other',
                   'mission_procurement', 'mission_type_other', 'mission_type',
                   'event_damage', 'event_injuries', 'event_org', 'event_org_other',
                   'event_state', 'event_location', 'event_time', 'event_date',
                   'public_yn', 'sequence_number', 'fiscal_year', 'unitid', 'safecomid',
                   'id']
document_id_col = 'id'
csv_file_name = smart_nlp_path+"input data/safecom-2011-present-NASA.xlsx" 

file = smart_nlp_path+r"/output data/test_safecom_topics-Sep-21-2021/preprocessed_data.csv"
safecom = Topic_Model_plus(document_id_col=document_id_col, extra_cols=extra_cols, list_of_attributes=list_of_attributes, combine_cols=False)
safecom.extract_preprocessed_data(file)
preprocessed_df = safecom.data_df

Preprocessed data extracted from:  C:\Users\srandrad\smart_nlp\//output data/test_safecom_topics-Sep-21-2021/preprocessed_data.csv


In [5]:
hazard_file = smart_nlp_path+r"/output data/test_safecom_topics-Sep-21-2021/hazard_interpretation_SAFECOM_V0.xlsx"
results_file = smart_nlp_path+r"/output data/test_safecom_topics-Sep-21-2021/lda_results.xlsx"

In [6]:
frequency_new, docs = identify_docs_per_hazard(hazard_file, preprocessed_df, results_file, "narr_public", 'fiscal_year', "id")

In [7]:
print(frequency_new)

{'Drone Incursion': {'2020': 39, '2015': 19, '2021': 19, '2018': 26, '2019': 14, '2016': 35, '2017': 41, '2014': 0}, 'Water Drop': {'2020': 157, '2015': 129, '2021': 119, '2018': 131, '2019': 65, '2016': 124, '2017': 120, '2014': 0}, 'Hydraulic Fluid Issues': {'2020': 25, '2015': 19, '2021': 23, '2018': 35, '2019': 12, '2016': 35, '2017': 28, '2014': 0}}
